In [134]:
from pathlib import Path
import pandas as pd

In [205]:
def grouped_results(merged_df, agg, target_column, join_symbol="±"):
    formating = lambda x: "{:.2f}".format(x)
    grouped_mean_formatted = merged_df.groupby(agg)[target_column].mean().map(formating)
    grouped_std_formatted = merged_df.groupby(agg)[target_column].std().map(formating)
    result_df = pd.concat([grouped_mean_formatted, grouped_std_formatted], axis=1)
    result_df.columns = ["Mean", "Std"]

    # Combine mean and std columns to display mean ± std
    result_df["Mean ± Std"] = result_df["Mean"] + f"{join_symbol}" + result_df["Std"]

    result_df["Mean ± Std"] = result_df["Mean ± Std"].map(lambda x: f"${x}$")
    return result_df


def compute_diffs(merged_df, merge_on=None, metric="avg_acc_tag"):
    if not merge_on:
        merge_on = ["num_tasks", "approach", "dataset", "seed"]
    reg_false_df = merged_df[merged_df["reg_layers"] == False]
    reg_true_df = merged_df[merged_df["reg_layers"] == True]

    # Merge the two dataframes based on 'num_classes', 'approach', 'dataset', and 'seed'
    merged_reg_diff = pd.merge(
        reg_false_df,
        reg_true_df,
        on=merge_on,
        suffixes=("_false", "_true"),
        how="inner",
    )
    merged_reg_diff[f"{metric}_diff"] = (
        merged_reg_diff[f"{metric}_true"] - merged_reg_diff[f"{metric}_false"]
    )
    return merged_reg_diff


def get_merged_df(paths):
    dfs = [pd.read_csv(path) for path in paths]

    merged_df = pd.concat(dfs, axis=0)
    return merged_df

# Big tables

In [206]:
root = Path("csvs")
convnext = root / "resnet34_skips"
paths = list(convnext.glob("cifar*.csv"))


merged_df = get_merged_df(paths)

In [207]:
agg = ["num_tasks", "dataset", "approach", "reg_layers"]
grouped_results(merged_df, agg, "avg_acc_tag", join_symbol="\pm")

Mean   Std      Mean ± Std
num_tasks dataset        approach   reg_layers                             
5         cifar100_fixed ewc        False       23.10  0.63  $23.10\pm0.63$
                                    True        33.50  0.47  $33.50\pm0.47$
                         finetuning False       20.55  0.42  $20.55\pm0.42$
                                    True        27.71  1.14  $27.71\pm1.14$
                         lwf        False       39.26  0.29  $39.26\pm0.29$
                                    True        50.30  0.74  $50.30\pm0.74$
                         replay     False       40.63  1.09  $40.63\pm1.09$
                                    True        43.62  0.14  $43.62\pm0.14$
10        cifar100_fixed ewc        False       12.22  0.14  $12.22\pm0.14$
                                    True        17.58  0.37  $17.58\pm0.37$
                         finetuning False       11.54  0.13  $11.54\pm0.13$
                                    True        13.58  0.18  $13.58\pm0.18$
                         lwf        False       25.45  1.18  $25.45\pm1.18$
                                    True        37.63  0.50  $37.63\pm0.50$
                         replay     False       35.42  2.84  $35.42\pm2.84$
                                    True        37.08  0.11  $37.08\pm0.11$
20        cifar100_fixed ewc        False        4.91  2.91   $4.91\pm2.91$
                                    True        12.23  0.78  $12.23\pm0.78$
                         finetuning False        5.03  0.75   $5.03\pm0.75$
                                    True         7.90  0.48   $7.90\pm0.48$
                         lwf        False       16.89  1.03  $16.89\pm1.03$
                                    True        20.68  0.64  $20.68\pm0.64$
                         replay     False       31.60  1.89  $31.60\pm1.89$
                                    True        32.45  0.28  $32.45\pm0.28$

In [208]:
agg = ["num_tasks", "dataset", "approach"]
merged_reg_diff = compute_diffs(merged_df)
grouped_results(merged_reg_diff, agg, "avg_acc_tag_diff", join_symbol="\pm")

Mean   Std      Mean ± Std
num_tasks dataset        approach                               
5         cifar100_fixed ewc         10.41  1.07  $10.41\pm1.07$
                         finetuning   7.16  1.11   $7.16\pm1.11$
                         lwf         11.04  0.51  $11.04\pm0.51$
                         replay       2.99  1.15   $2.99\pm1.15$
10        cifar100_fixed ewc          5.36  0.28   $5.36\pm0.28$
                         finetuning   2.04  0.31   $2.04\pm0.31$
                         lwf         12.18  0.80  $12.18\pm0.80$
                         replay       1.66  2.79   $1.66\pm2.79$
20        cifar100_fixed ewc          7.31  3.30   $7.31\pm3.30$
                         finetuning   2.87  1.15   $2.87\pm1.15$
                         lwf          3.51  0.44   $3.51\pm0.44$
                         replay       0.85  1.79   $0.85\pm1.79$

# Big task

In [214]:
root = Path("csvs")
convnext = root / "big_task"
paths = list(convnext.glob("*.csv"))


merged_df = get_merged_df(paths)

In [215]:
merged_df

,reg_layers,real_name,wavg_acc_tag,misc.seed,data.datasets
0,True,replay,35.87,0,c
1,True,lwf,54.09,0,c
2,True,finetuning,29.37,0,c
3,True,ewc,39.28,0,c
4,False,lwf,44.83,0,c
5,False,ewc,30.41,0,c
6,False,replay,29.15,0,c
7,False,finetuning,23.22,0,c


In [219]:
agg = ["real_name", "reg_layers"]
grouped_results(merged_df, agg, "wavg_acc_tag", join_symbol="\pm ")

Mean  Std      Mean ± Std
real_name  reg_layers                            
ewc        False       30.41  nan  $30.41\pm nan$
           True        39.28  nan  $39.28\pm nan$
finetuning False       23.22  nan  $23.22\pm nan$
           True        29.37  nan  $29.37\pm nan$
lwf        False       44.83  nan  $44.83\pm nan$
           True        54.09  nan  $54.09\pm nan$
replay     False       29.15  nan  $29.15\pm nan$
           True        35.87  nan  $35.87\pm nan$

In [221]:
agg = ["real_name"]
merged_reg_diff = compute_diffs(
    merged_df,
    merge_on=["real_name", "misc.seed", "data.datasets"],
    metric="wavg_acc_tag",
)
grouped_results(merged_reg_diff, agg, "wavg_acc_tag_diff", join_symbol="\pm ")

,Mean,Std,Mean ± Std
real_name,,,
ewc,8.87,nan,$8.87\pm nan$
finetuning,6.15,nan,$6.15\pm nan$
lwf,9.26,nan,$9.26\pm nan$
replay,6.72,nan,$6.72\pm nan$
